# Modules

In [ ]:
# import sys; sys.path.append("../modules")
from vkdiagram import *

#import matplotlib.pyplot as plt
#import networkx as nx

# Drawing virtual knot diagrams

In [ ]:
class Node():
    """A node class for A* Pathfinding"""

    def __init__(self, parent=None, position=None):
        self.parent = parent # 親ノードの設定
        self.position = position # (row, column)のタプル ※row：行、column：列

        self.g = 0
        self.h = 0
        self.f = 0

    def __eq__(self, other):
        # node 同士の比較演算子(==)を使用できるように
        return self.position == other.position

In [ ]:
#nw = Nanoword('ABCBCA', [Letter('A','a+'), Letter('B','b+'), Letter('C','a-')]) #('CABADBDC', [Letter('A','b-'), Letter('B','a-'), Letter('C','b-'), Letter('D','a+')])
nw = Nanoword.random_generator(4) #
print(nw, nw.alphabet); diag = Diagram(nw)

marr = diag.matrix
alph = nw.alphabet
nwinfo = nw.get_full_info()


for j in range(nw.size):
    d, d_next = nwinfo[j], nwinfo[(j+1)%nw.size]
    ltr, ltr_next = d['letter'], d_next['letter']
    lind, lind_next = alph.index(ltr), alph.index(ltr_next)
    sposi, gposi = 3*lind+2, 3*lind_next+2
    b_or_t = 't' if (ltr.sign.ab == 'a') else 'b'
    b_or_t_next = 'b' if (ltr_next.sign.ab == 'a') else 't'
    #---
    start = Node(position=(sposi, sposi))
    start.g = start.h = start.f = 0
    goal = Node(position=(gposi, gposi))
    goal.g = goal.h = goal.f = 0
    #-----
    closed_list = []
    open_list = []
    #-----
    # # The first and last node
    # if d['is_first']:
    #     dhori = 1
    #     dvirt = 0
    # else: 
    #     dhori = 0
    #     dvirt = 1 if b_or_t == 'b' else -1
    # current = Node(position=(sposi + dvirt, sposi + dhori), parent=start)
    # marr[current.position] = lind + 1
    current = start
    current.g = current.h = current.f = 0
    open_list.append(current)
    #---
    if d_next['is_first']:
        d_col = -1
        d_row = 0
    else: 
        d_col = 0
        d_row = 1 if b_or_t_next == 'b' else -1
    end_node = Node(position=(gposi + d_row, gposi + d_col), parent=None)
    #-----
    # approaching goal by the A* algorithm
    while len(open_list) > 0:
        # Get the current node
        current_node = open_list[0]
        current_index = 0
        for index, item in enumerate(open_list):
            # オープンリストの中でF値が一番小さいノードを選ぶ
            if item.f < current_node.f:
                current_node = item
                current_index = index

        # Pop current off open list, add to closed list
        open_list.pop(current_index)
        closed_list.append(current_node)

        # Found the goal
        if current_node == end_node:
            goal.parent = current_node
            path = [goal]
            current = current_node
            while current != start:
                path.append(current)
                current = current.parent
            #---
            path = path[::-1]
            for k, node in enumerate(path):
                if k == len(path)-1:
                    continue
                label = ''
                for dff in [-1, 1]:
                    if dff == -1 and k == 0:
                        if d['is_first']:
                            label += 'l'
                        else:
                            label += 't' if b_or_t == 'b' else 'b'
                    else:
                        dffs = (path[k+dff].position[0]-node.position[0], path[k+dff].position[1]-node.position[1])
                        match dffs:
                            case (0, 1):
                                label += 'r'
                            case (0, -1):
                                label += 'l'
                            case (1, 0):
                                label += 'b'
                            case (-1, 0):
                                label += 't'
                match label:
                    case 'rl' | 'lr':
                        marr[node.position] += 0b01
                    case 'rb' | 'br':
                        marr[node.position] = 0b1100 + 0b0000 + 0b0000
                    case 'rt' | 'tr':
                        marr[node.position] = 0b1100 + 0b0000 + 0b0001
                    case 'lb' | 'bl':
                        marr[node.position] = 0b1100 + 0b0010 + 0b0000
                    case 'lt' | 'tl':
                        marr[node.position] = 0b1100 + 0b0010 + 0b0001
                    case 'tb' | 'bt':
                        marr[node.position] += 0b10
                    case _:
                        print(f"What a f**k!:{label=}")
            # print(f"{open_list=}")
            break
            # open_list = []
            # continue

        # Generate children
        children = []
        for new_position in [(0, -1), (0, 1), (-1, 0), (1, 0)]:
            # Make sure limited directions
            if current_node == start:
                # print(f"{current_node.position=}, {new_position=}, {d['is_first'] and new_position != (0, 1)}")
                if d['is_first']:
                    if new_position != (0, 1):
                        continue
                else:
                    if b_or_t == 'b' and new_position != (1, 0):
                        continue
                    elif b_or_t == 't' and new_position != (-1, 0):
                        continue
            else:
                v = marr[current_node.position]
                if (v == 0b01 and new_position[1] != 0) or (v == 0b10 and new_position[0] == 0):
                    continue
            # Get node position
            # print(f"{new_position=}")
            node_position = (current_node.position[0] + new_position[0], current_node.position[1] + new_position[1])
            # Make sure within range
            if node_position[0] > (len(marr) - 1) or node_position[0] < 0 or node_position[1] > (len(marr) - 1) or node_position[1] < 0:
                continue
            # Make sure walkable terrain
            if (current_node != start) and (node_position != end_node.position):
                if not marr[node_position] == 0:
                    continue
            # Create new node
            new_node = Node(current_node, node_position)
            children.append(new_node)

        if not children:
            print(f"{current_node.position=}, {start.position=}, {d['is_first']=}, {[n.position for n in children]} \n")
        # Loop through children
        for child in children:
            # Child is on the closed list
            if len([closed_child for closed_child in closed_list if closed_child == child])>0:
                continue

            # Create the f, g, and h values
            child.g = current_node.g + 1
            if current != start:
                if current_node.parent.position[0] == current_node.position[0]:
                    child.g += 1 if current_node.position[0] != child.position[0] else 0
                else:
                    child.g += 1 if current_node.position[1] != child.position[1] else 0
            # H は （現在位置 - エンド位置)の2乗
            child.h = ((child.position[0] - end_node.position[0]) ** 2) + ((child.position[1] - end_node.position[1]) ** 2)
            # F = G + H
            child.f = child.g + child.h

            # Child is already in the open list
            if [open_node for open_node in open_list if child.position == open_node.position and child.g > open_node.g]:
                continue

            # Add the child to the open list
            open_list.append(child)

print(marr); 
diag.show()

In [ ]:
print(marr)